In [1]:
import pandas as pd
import string

from Utils.labels import *
from Utils.extract_words import *
from Utils.evaluation import *
import nltk

In [2]:
df = pd.read_json("Data.json")
with open('./Triggers_Cues/uncertainty.txt', 'r') as file:
    uncertainty_cues = [line.strip() for line in file]

with open('./Triggers_Cues/negations.txt', 'r') as file:
    negations_cues = [line.strip() for line in file]

In [3]:
import string

def identify_scope(text, End_sentence, End_word):

    sentences_of_text = []
    sentence = []

    negation_found    = False
    uncertainti_found = False

    all_words = list(get_nextWord(text, End_word))
    for _, token in all_words:
        tag = (token, "other")

        if token in negations_cues:
            negation_found    = True
            tag = (token, "NEG")

        elif token in uncertainty_cues:
            uncertainti_found = True
            tag = (token, "UNC")

        sentence.append(tag)

        if any(char in token for char in End_sentence):  #Look if any char in the token contains an end of sentence character.

            if (negation_found):
              sentence = [(tag[0], "NSCO") if tag[1] == 'other' else tag for tag in sentence]

            elif (uncertainti_found):
              sentence = [(tag[0], "USCO") if tag[1] == 'other' else tag for tag in sentence]

            sentences_of_text.extend(sentence)

            sentence          = []
            negation_found    = False
            uncertainti_found = False
    # Devolver la lista de oraciones con la negación y su scope
    return sentences_of_text

In [197]:
End_sentence   = '!.?,;:'   #These are possible punctuations that can appear in the words return by get_nextWord
Difference     = set(string.punctuation) - set(End_sentence)

In [198]:
End_word = ''             #What we consider as a delimeter for defining what is a word
for char in Difference:
    End_word += char

In [199]:
y_pred = [identify_scope(df["data"][i]["text"], End_sentence, End_word) for i in range(len(df))]
y_true = [label_text(df, i, End_word) for i in range(len(df))]

In [200]:
y_pred_ext = []
y_true_ext = []
for pred, true in zip(y_pred, y_true):
    len_diff = len(true) - len(pred)
    if len_diff > 0:
        pred.extend([(' ', 'other')] * len_diff)
    if len_diff < 0:
        true.extend([(' ', 'other')] * (abs(len_diff)))
    y_pred_ext.append(pred)
    y_true_ext.append(true)

In [194]:
def sent2labels(sent):
    return [label for _, label in sent]

def sent2tokens(sent):
    return [token for token, _, in sent]


In [201]:
y_true_labels = [sent2labels(sent) for sent in y_true_ext]
y_pred_labels = [sent2labels(sent) for sent in y_pred_ext]

In [202]:
print(bio_classification_report(y_true_labels, y_pred_labels))

              precision    recall  f1-score   support

         NEG       0.71      0.88      0.78      4862
        NSCO       0.32      0.87      0.47     13915
         UNC       0.53      0.51      0.52       743
        USCO       0.18      0.65      0.29      1846
       other       0.99      0.86      0.92    219399

   micro avg       0.86      0.86      0.86    240765
   macro avg       0.55      0.75      0.60    240765
weighted avg       0.94      0.86      0.88    240765
 samples avg       0.86      0.86      0.86    240765

